In [32]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics

%matplotlib inline

In [33]:
def return_varlog(time_series):
    lag = np.roll(time_series, 1)
    lag[0] = 0
    ret = np.log(time_series/lag)
    ret[0] = 0
    return ret

In [15]:
time_series = [51.83,51.77,51.3,52.09,51.91,51.09,49.27,50.5,50.05,50.21,50.28,50.75,51.67,51.05]
time_series

[51.83,
 51.77,
 51.3,
 52.09,
 51.91,
 51.09,
 49.27,
 50.5,
 50.05,
 50.21,
 50.28,
 50.75,
 51.67,
 51.05]

In [29]:
lag = np.roll(time_series, 1)
lag[0] = 0
ret = np.log(time_series/lag)
ret[0] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
lag.tolist()

[0.0,
 51.83,
 51.77,
 51.3,
 52.09,
 51.91,
 51.09,
 49.27,
 50.5,
 50.05,
 50.21,
 50.28,
 50.75,
 51.67]

In [31]:
ret.tolist()

[0.0,
 -0.0011583012878050875,
 -0.0091200787370862,
 0.01528223958037649,
 -0.0034615419179625994,
 -0.01592266649643087,
 -0.03627340678683487,
 0.024657959725442262,
 -0.008950830520084557,
 0.003191704285384607,
 0.0013931736754325171,
 0.009304234199850095,
 0.01796572437719658,
 -0.012071797688418776]

In [36]:
return_varlog(time_series).tolist()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


[0.0,
 -0.0011583012878050875,
 -0.0091200787370862,
 0.01528223958037649,
 -0.0034615419179625994,
 -0.01592266649643087,
 -0.03627340678683487,
 0.024657959725442262,
 -0.008950830520084557,
 0.003191704285384607,
 0.0013931736754325171,
 0.009304234199850095,
 0.01796572437719658,
 -0.012071797688418776]

In [38]:
data = pd.read_csv('datasets/data_alpha.csv')
data

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,28/03/2019,16.75,37.25,32.07,48.02,30.8333,35.1833,26.30,6.8732,34.19,...,24.72,11.77,39.3253,23.700,9.68,49.30,47.21,4.18,18.05,27.21
1,29/03/2019,16.83,38.27,32.12,48.72,31.5417,35.7833,26.52,6.9126,35.87,...,24.84,11.79,39.3253,23.500,10.04,50.93,47.50,4.22,18.00,26.60
2,01/04/2019,16.82,38.94,32.58,49.20,31.6000,36.0700,26.38,6.9716,36.22,...,24.88,11.80,39.1756,23.550,10.22,52.60,47.62,4.27,18.07,26.70
3,02/04/2019,16.77,38.80,32.66,48.77,31.3000,35.5900,27.25,7.0994,36.20,...,24.80,11.67,38.9260,23.245,9.70,51.63,47.07,4.55,18.25,27.19
4,03/04/2019,16.84,38.96,31.56,48.25,30.9000,35.1900,27.79,6.9814,36.82,...,24.67,11.77,40.1337,22.900,9.69,51.78,47.51,4.38,18.47,27.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,05/06/2020,14.05,21.16,49.07,35.20,20.4500,22.3000,28.49,13.0100,60.24,...,28.93,14.60,20.7600,18.680,6.90,54.61,47.77,14.85,44.33,38.56
296,08/06/2020,14.60,27.35,51.72,36.39,21.4300,23.4700,28.40,13.0500,66.02,...,28.94,14.44,21.0300,19.130,7.45,54.78,48.54,15.50,45.80,37.15
297,09/06/2020,14.32,25.78,51.07,35.79,21.2100,22.9500,27.62,12.5900,65.58,...,28.81,14.48,21.1100,19.270,7.30,54.99,49.00,14.97,45.81,36.64
298,10/06/2020,13.96,23.50,50.74,34.21,20.3600,21.8800,26.85,12.7700,63.36,...,29.05,14.30,21.4900,18.620,6.89,54.20,49.59,14.95,44.99,34.90


In [43]:
data['ABEV3'].isnull().sum()

0

In [47]:
test=[]
for col in data.columns:
    su = data[col].isnull().sum()
    if(su > 0):
        test.append([col, su])
test

[['NTCO3', 240]]